## Gold analysis
Se il massimo di mercoledi, e' inferiore a quello di lunedi, vedrai il minimo di mercoledi rivisitato giovedi (rivisitato=giovedi il prezzo crossa dall'alto o dal basso il minimo di mercoledi)

### Step da fare:
- Calcolare dei massimi e minimi giornalieri
    - bisogna fare resampling dei dati (in modo da ottenere candele da 1 giorno)
- Categorizzare ogni giorno della settimana (bisogna creare una colonna che dice che giorno e')
- Controllare se il max di mercoledi e' inferiore a quello di lunedi'
    - salva il minimo di mercoledi'
    - controlla se il range di giovedi include il minimo di mercoledi'.

### Readint the CSV file and converting it to a parquet one:

In [1]:
# #import the libraries
# import cudf
# import dask
# import dask.dataframe as dd
# #set the enviroment to cuDF so we use the GPU
# dask.config.set({"dataframe.backend": "cudf"})
# #----------------------------------------------

# xau1D = dd.read_csv('/home/edoardocame/Desktop/python_dir/xauusd-d1-bid-2014-01-01-2024-12-11T23.csv')
# xau1D['timestamp'] = dd.to_datetime(xau1D['timestamp'])
# xau1D = xau1D.set_index('timestamp', sorted=True)
# xau1D['weekday'] = xau1D.index.to_series().dt.weekday
# xau1D.head()

### Using parquet file:

In [2]:
#import the libraries
import dask
import dask.dataframe as dd
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
cluster = LocalCUDACluster()
client = Client(cluster)
client
#----------------------------------------------
dask.config.set({"dataframe.backend": "cudf"})


df = dd.read_parquet('/home/edoardocame/Desktop/python_dir/xauusd1D.parquet')
df['returns'] = df['close'].diff() / df['close'].shift(1)
df['week'] = df.index.dt.isocalendar().week
df['year'] = df.index.dt.isocalendar().year
df.head()

,open,high,low,close,volume,weekday,returns,week,year
timestamp,,,,,,,,,
2014-01-01,1203.612,1205.883,1202.302,1205.883,0.2705,2,<NA>,1,2014
2014-01-02,1205.913,1230.773,1204.893,1223.710,27.3592,3,0.014783358,1,2014
2014-01-03,1223.687,1240.153,1223.297,1236.683,26.3572,4,0.010601368,1,2014
2014-01-05,1236.983,1238.353,1233.842,1234.042,0.3280,6,-0.002135551,1,2014
2014-01-06,1234.042,1248.342,1214.626,1237.665,26.1419,0,0.002935881,2,2014


In [3]:
# Create separate dataframes for each day we need
# Group by year and week, then get the first occurrence (should be only one per day anyway)
monday_data = df[df['weekday'] == 0].groupby(['year', 'week'])['high'].first()
wednesday_data = df[df['weekday'] == 2].groupby(['year', 'week'])['high'].first()
thursday_data = df[df['weekday'] == 3].groupby(['year', 'week'])[['high', 'low']].first()

weekly_analysis = dd.concat([monday_data.rename('mon_high'), wednesday_data.rename('wed_high'), thursday_data], axis=1)
weekly_analysis.head()

/home/edoardocame/miniconda3/envs/rapids-23.12/lib/python3.10/site-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


mon_high  wed_high      high       low
year week                                        
2014 1         <NA>  1205.883  1230.773  1204.893
     2     1248.342  1231.888  1231.463  1223.475
     3     1255.042  1244.012  1245.198  1236.328
     4     1258.205  1243.557  1265.433  1231.523
     5     1276.438  1270.142  1267.677  1238.043

In [5]:
weekly_analysis['wed_lower_than_mon'] = weekly_analysis['wed_high'] < weekly_analysis['mon_high']

weekly_analysis['thurs_crosses_wed'] = (
    (weekly_analysis['wed_lower_than_mon']) & 
    (weekly_analysis['low'] <= weekly_analysis['wed_high']) & 
    (weekly_analysis['high'] >= weekly_analysis['wed_high'])
)

weekly_analysis.head()

mon_high  wed_high      high       low wed_lower_than_mon  \
year week                                                              
2014 1         <NA>  1205.883  1230.773  1204.893               <NA>   
     2     1248.342  1231.888  1231.463  1223.475               True   
     3     1255.042  1244.012  1245.198  1236.328               True   
     4     1258.205  1243.557  1265.433  1231.523               True   
     5     1276.438  1270.142  1267.677  1238.043               True   

          thurs_crosses_wed  
year week                    
2014 1                 <NA>  
     2                False  
     3                 True  
     4                 True  
     5                False

In [11]:
print(weekly_analysis['thurs_crosses_wed'].sum().compute())
print(len(weekly_analysis['thurs_crosses_wed']))

123
572


In [7]:
client.shutdown()